In [3]:
import pythia8 

import awkward as ak
import numpy as np
import pickle as pkl
import hist 
import matplotlib.pyplot as plt
import uproot

import time

from numba import njit

In [4]:
RAW_DATA_DIR = '/Users/ravikoka/repos/z_plus_hf/feasibility/data/'

In [10]:
data1 = process_event(3, 4, pythia.event)
pythia.next()
data2 = process_event(3, 5, pythia.event)
pythia.next()
data3 = process_event(3, 6, pythia.event)

In [19]:
data1['pdg'][0]

<Array [2212, 2212, 2, 21, 2, 21, ..., 22, 22, 22, 22, 22] type='1263 * int64'>

In [20]:
data1['px'][0]

<Array [0, 0, 0, 0, ..., -0.273, -0.0342, -0.0839] type='1263 * float64'>

In [21]:
ak.flatten(data1['mother_list'])

<Array [[], [], [7], ..., [1218], [1229], [1229]] type='1263 * var * int64'>

In [17]:
data1['pdg'][0][8]

np.int64(2)

In [5]:
NUM_EVENTS = int(1e3) 
BATCH_SIZE = 500
NUM_BATCHES = NUM_EVENTS // BATCH_SIZE

pythia = pythia8.Pythia()

pythia.readString("Beams:eCM = 13600")
pythia.readString("HardQCD:all = on")
pythia.readString("PhaseSpace:pTHatMin = 20.") # check this

pythia.readString("WeakBosonAndParton:all = on")        
pythia.readString("WeakBosonExchange:all = on")
pythia.readString("WeakDoubleBoson:all = on")
# pythia.readString("WeakSingleBoson:all = on")
pythia.readString("WeakSingleBoson:ffbar2gmZ = on")
pythia.readString("WeakSingleBoson:ffbar2ffbar(s:gmZ) = on")

pythia.readString("WeakZ0:gmZ0mode = 2")


pythia.readString("SoftQCD:all = off")

pythia.init()


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.312      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 23 May 2024  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 20 Nov 2024 at 21:16:36    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

True

In [6]:
def process_event(batch_num, event_num, pythia_event):
    px = [] 
    py = []
    pz = []
    E = []
    pdg = []
    generator_status = []
    mother_lists = []
    daughter_lists = []
    is_final_state = []
    
    for i, particle in enumerate(pythia_event):
        if i == 0:
            continue 
        
        px.append(particle.px())
        py.append(particle.py())
        pz.append(particle.pz())
        E.append(particle.e())
        pdg.append(particle.id())
        generator_status.append(particle.status())
        mother_lists.append(particle.motherList())
        daughter_lists.append(particle.daughterList())
        is_final_state.append(particle.isFinal())
        
    particle_data = {
        "px": np.array(px),
        "py": np.array(py),
        "pz": np.array(pz),
        "E":  np.array(E),
        "pdg": np.array(pdg),
        "gen_status": np.array(generator_status),
        "mother_list": mother_lists, # check if motherlist is indexed with zero as first particle or row 0 in event record (event level info)
        "daughter_list": daughter_lists,
        "is_final": np.array(is_final_state),
        #"event_num" : event_num,
        #"batch_num" : batch_num,
    }
        
    return ak.Array([particle_data])
    #return particle_data

In [9]:
for batch_num in range(NUM_BATCHES):

    events = ak.Array([])
    for event_num in range(BATCH_SIZE):
        
        if not pythia.next():
            continue
        
        particle_data = process_event(batch_num, event_num, pythia.event)
        
        events = ak.concatenate([events, particle_data])
        # t0 = time.time()
        # events = ak.concatenate([events, [particle_data]])
        # t1 = time.time()
        
        #print(t1-t0)
        #print(events.type)
    
    print(f'batch num: {batch_num}, num events processed: {(batch_num + 1) * BATCH_SIZE}')
       
    with open(f'{RAW_DATA_DIR}pp_Z_production_13600_{batch_num}.pkl', 'wb') as out_file:
        pkl.dump(events, out_file)

batch num: 0, num events processed: 500
batch num: 1, num events processed: 1000


In [244]:
for batch_num in range(NUM_BATCHES):

    events = ak.Array([])
    for event_num in range(BATCH_SIZE):
        
        if not pythia.next():
            continue
        print(event_num)
        #particles = []
        px = []
        py = []
        pz = []
        E = []
        pdg = []
        for i, particle in enumerate(pythia.event):
            if i == 0:
                continue 
            
            px.append(particle.px())
            py.append(particle.py())
            pz.append(particle.pz())
            E.append(particle.e())
            pdg.append(particle.id())
        
        
            
        # particle_data = {
        #     "px": ak.Array(px),
        #     "py": ak.Array(py),
        #     "pz": ak.Array(pz),
        #     "E":  ak.Array(E),
        #     "pid": ak.Array(pdg),
        # }
        
        
        # particle_data = {
        #     "px": np.array(px),
        #     "py": np.array(py),
        #     "pz": np.array(pz),
        #     "E":  np.array(E),
        #     "pid": np.array(pdg),
        #     "event_num" : event_num,
        # } uncomment
        
        particle_data = process_event(batch_num, event_num, pythia.event)
        
            
            # particle_properties = ak.Array([
            #                                 {'px': particle.px(), 
            #                                 'py': particle.py(), 
            #                                 'pz': particle.pz(), 
            #                                 'E': particle.e(), 
            #                                 'pdg': particle.id(), 
            #                                 'gen_status': particle.status()}
            #                                 ])

            # particles.append(particle_properties)
        
        if not event_num % 100:
            print(f'event num: {event_num}')
            
        t0 = time.time()
        events = ak.concatenate([events, [particle_data]])#, axis=0)
        t1 = time.time()
        print(t1-t0)
        #print(i)
        #print(len(particles))

    with open(f'pp_Z_production_13600_{batch_num}.pkl', 'wb') as out_file:
        pkl.dump(events, out_file)
    
    # print(events)
    # with uproot.recreate(f'pp_Z_production_13600_{batch_num}.root') as out_file:
    #     out_file['events'] = events

0
event num: 0
0.08997416496276855
1
0.08512997627258301
2
0.17296719551086426
3
0.020764827728271484
4
0.1227262020111084
5
0.16061806678771973
6
0.07173466682434082
7
0.13840699195861816
8
0.1389172077178955
9
0.0913238525390625
10
0.1620018482208252
11
0.08627009391784668
12
0.11558198928833008
13
0.08737707138061523
14
0.1741938591003418
15
0.12558412551879883
16
0.16524887084960938
17
0.1474440097808838
18
0.1579279899597168
19
0.19356608390808105
20
0.09017610549926758
21
0.07291483879089355
22
0.1865980625152588
23
0.13842177391052246
24
0.06920313835144043
25
0.10329008102416992
26
0.18781590461730957
27
0.14739203453063965
28
0.08352327346801758
29
0.09165096282958984
30
0.040911197662353516
31
0.1715238094329834
32
0.0441739559173584
33
0.12653589248657227
34
0.11580181121826172
35
0.09698295593261719
36
0.22660303115844727
37
0.047934770584106445
38
0.13293194770812988
39
0.1679530143737793
40
0.20068812370300293
41
0.13605690002441406
42
0.1413278579711914
43
0.040184736251

KeyboardInterrupt: 

In [ ]:
data1 = process_event(3, 4, pythia.event)
pythia.next()
data2 = process_event(3, 5, pythia.event)
pythia.next()
data3 = process_event(3, 6, pythia.event)

In [241]:
events['px']

<Array [[0, 0, 0, ..., 0.134, 0.0569, -0.0662], ...] type='500 * var * float64'>

In [94]:
events[2]

<Record {px: [0, ...], py: [0, ...], ...} type='{px: var * float64, py: var...'>

In [89]:
events

<Array [{px: [0, ...], py: [...], ...}, ...] type='100 * {px: var * float64...'>

In [33]:
for batch_num in range(NUM_BATCHES):

    events = ak.Array([])
    for event_num in range(BATCH_SIZE):
        
        if not pythia.next():
            continue
        
        particles = []
        for i, particle in enumerate(pythia.event):
            if i == 0:
                continue 
            
            particle_properties = ak.Array([
                                            {'px': particle.px(), 
                                            'py': particle.py(), 
                                            'pz': particle.pz(), 
                                            'E': particle.e(), 
                                            'pdg': particle.id(), 
                                            'gen_status': particle.status()}
                                            ])

            particles.append(particle_properties)
        
        if not event_num % 100:
            print(f'event num: {event_num}')
            
        events = ak.concatenate([events, [particles]], axis=0)
        #print(i)
        #print(len(particles))

    with open(f'pp_Z_production_13600_{batch_num}.pkl', 'wb') as out_file:
        pkl.dump(events, out_file)

event num: 0
event num: 0
event num: 0
event num: 0
event num: 0
event num: 0
event num: 0
event num: 0
event num: 0
event num: 0


In [84]:
with uproot.open('pp_Z_production_13600_9.root') as file:
    events = file['events'].arrays()

In [86]:
events

<Array [] type='0 * {"": float64}'>

In [72]:
ak.flatten(events['px'])

AxisError: axis=1 exceeds the depth of this array (1)

In [67]:
with open('pp_Z_production_13600_9.pkl', 'rb') as in_file:
    read = pkl.load(in_file)

In [76]:
np.sum(ak.count(read['px'], axis=-1) == 1575)

np.int64(0)

In [19]:
np.sum(ak.flatten(read['pdg']) == 2)

np.int64(31234)

In [ ]:
import multiprocessing
from pythia8 import Pythia

def generate_events(pythia_instance, n_events):
    """Generates a specified number of events using a Pythia instance."""
    pythia_instance.init()
    for i in range(n_events):
        pythia_instance.next()
        # Process the event here (e.g., fill histograms)
        # ...
        px = [] 
        py = []
        pz = []
        E = []
        pdg = []
        
        for i, particle in enumerate(pythia.event):
            if i == 0:
                continue 
            
            px.append(particle.px())
            py.append(particle.py())
            pz.append(particle.pz())
            E.append(particle.e())
            pdg.append(particle.id())
            
        particle_data = {
            "px": np.array(px),
            "py": np.array(py),
            "pz": np.array(pz),
            "E":  np.array(E),
            "pid": np.array(pdg),
        }
        
        return particle_data
    

if __name__ == "__main__":
    n_processes = 4  # Number of processes to use
    n_events_per_process = 1000  # Number of events per process

    # Create a list of Pythia instances
    pythia_instances = [Pythia() for _ in range(n_processes)]

    # Set Pythia settings for each instance if needed
    for pythia in pythia_instances:
        pythia.readString("Beams:idA = 2212")
        pythia.readString("Beams:idB = 2212")
        pythia.readString("Beams:eCM = 13000")

    # Create a pool of processes
    pool = multiprocessing.Pool(processes=n_processes)

    # Generate events in parallel
    pool.starmap(generate_events, zip(pythia_instances, [n_events_per_process] * n_processes))

    pool.close()
    pool.join()